In [2]:
import numpy as np
import pandas as pd

In [3]:
transactions = pd.read_csv('data-files/trx_data.csv')

In [4]:
transactions.info()  
transactions.head(10)  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62483 entries, 0 to 62482
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   customerId  62483 non-null  int64 
 1   products    62483 non-null  object
dtypes: int64(1), object(1)
memory usage: 976.4+ KB


,customerId,products
0,0,20
1,1,2|2|23|68|68|111|29|86|107|152
2,2,111|107|29|11|11|11|33|23
3,3,164|227
4,5,2|2
5,6,144|144|55|266
6,7,135|206|259
7,8,79|8|8|48
8,9,102|2|2|297
9,10,84|77|290|260


In [5]:
products = transactions['products'].str.split('|')
products   #문자열

0                                             [20]
1        [2, 2, 23, 68, 68, 111, 29, 86, 107, 152]
2               [111, 107, 29, 11, 11, 11, 33, 23]
3                                       [164, 227]
4                                           [2, 2]
                           ...                    
62478                       [103, 103, 8, 48, 126]
62479    [124, 37, 37, 78, 124, 124, 37, 8, 35, 8]
62480                                         [24]
62481                                        [167]
62482                                        [150]
Name: products, Length: 62483, dtype: object

In [6]:
transactions['products']=products.map(lambda x: [int(p) for p in x])  # map은 1:1 변환 처리때 사용 //  #문자열(str) -> 숫자(int)로 바꿈

In [7]:
print(transactions.iloc[1,1])
print(type(transactions.iloc[1,1]))
print(type(transactions.iloc[1,1][0]))
transactions.head()

[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]
<class 'list'>
<class 'int'>


,customerId,products
0,0,[20]
1,1,"[2, 2, 23, 68, 68, 111, 29, 86, 107, 152]"
2,2,"[111, 107, 29, 11, 11, 11, 33, 23]"
3,3,"[164, 227]"
4,5,"[2, 2]"


In [13]:
data_list = []
for idx, transaction in transactions.iterrows():
    for product in transaction['products']:
        data_list.append({'customerId': transaction['customerId'], 'productId': product})

In [16]:
data_list[:10]

[{'customerId': 0, 'productId': 20},
 {'customerId': 1, 'productId': 2},
 {'customerId': 1, 'productId': 2},
 {'customerId': 1, 'productId': 23},
 {'customerId': 1, 'productId': 68},
 {'customerId': 1, 'productId': 68},
 {'customerId': 1, 'productId': 111},
 {'customerId': 1, 'productId': 29},
 {'customerId': 1, 'productId': 86},
 {'customerId': 1, 'productId': 107}]

In [17]:
data = pd.DataFrame(data_list)

In [18]:
data.head()

,customerId,productId
0,0,20
1,1,2
2,1,2
3,1,23
4,1,68


In [19]:
data['purchase_count'] = np.ones(len(data))
data.head()

,customerId,productId,purchase_count
0,0,20,1.0
1,1,2,1.0
2,1,2,1.0
3,1,23,1.0
4,1,68,1.0


In [28]:
data2 = data.groupby(['customerId','productId'], as_index=False).count()   # as_index=False 는 reset_index와 같은 기능임.

In [29]:
# data2.columns = ['userid','itemid','rating']
data2.head()

,customerId,productId,purchase_count
0,0,1,2
1,0,13,1
2,0,19,3
3,0,20,1
4,0,31,2


In [31]:
data2['purchase_count'].describe()

count    133585.000000
mean          1.583097
std           1.116338
min           1.000000
25%           1.000000
50%           1.000000
75%           2.000000
max          75.000000
Name: purchase_count, dtype: float64

In [32]:
from surprise.reader import Reader
from surprise import Dataset

In [33]:
reader = Reader()
ds = Dataset.load_from_df(data2, reader)

In [36]:
df=pd.DataFrame(ds.raw_ratings, columns=['custid','product','count','id'])

In [37]:
df.head()

,custid,product,count,id
0,0,1,2.0,None
1,0,13,1.0,None
2,0,19,3.0,None
3,0,20,1.0,None
4,0,31,2.0,None


In [38]:
df.drop('id', axis=1, inplace=True)

In [39]:
df.head()

,custid,product,count
0,0,1,2.0
1,0,13,1.0
2,0,19,3.0
3,0,20,1.0
4,0,31,2.0


In [40]:
df2=df.set_index(['custid','product'])
df2.head()

count
custid product       
0      1          2.0
       13         1.0
       19         3.0
       20         1.0
       31         2.0

In [42]:
df3=df2.unstack()
df3.head()

count                                        ...                       \
product   0    1    2   3   4   5   6   7   8   9    ... 290 291 292  293 294   
custid                                               ...                        
0         NaN  2.0  NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN  NaN NaN   
1         NaN  NaN  6.0 NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN  1.0 NaN   
2         NaN  NaN  NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN  NaN NaN   
3         NaN  NaN  NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN  NaN NaN   
4         NaN  NaN  NaN NaN NaN NaN NaN NaN NaN NaN  ... NaN NaN NaN  NaN NaN   

                              
product 295  296 297 298 299  
custid                        
0       NaN  NaN NaN NaN NaN  
1       NaN  1.0 NaN NaN NaN  
2       NaN  NaN NaN NaN NaN  
3       NaN  NaN NaN NaN NaN  
4       NaN  NaN NaN NaN NaN  

[5 rows x 300 columns]

In [43]:
df3.shape

(24429, 300)

In [49]:
df3.iloc[:10, :20].fillna('') 

count                                                         
product    0  1  2  3  4  5  6  7  8  9  10 11 12 13 14 15 16 17 18 19
custid                                                                
0              2                                   1                 3
1                 6                                                   
2                                            3                        
3                                                                     
4                                                        2            
5                 2                                               3   
6                                                                     
7                                                                     
8                                   3  1              2           1   
9           3  2  6                                   1  1     2  6

In [50]:
from surprise.model_selection import KFold
from surprise import BaselineOnly, accuracy
import numpy as np

In [52]:
bsl_option = {'method':'als',       #als, sgd 두가지중 als가 좀더 최적화 모델이다. 
              'n_epochs':5,         #반복 횟수
              'reg_u':12,           #사용자 가중치
              'reg_i':5             #상품 가중치
             }

model = BaselineOnly(bsl_option)

np.random.seed(42)
accuracies = np.zeros(3)
cv=KFold(3)    
for i, (trainset, testset) in enumerate(cv.split(ds)):
    model.fit(trainset)
    predictions = model.test(testset)
    accuracies[i] = accuracy.rmse(predictions, verbose=True)
    
accuracies.mean()

Estimating biases using als...
RMSE: 1.0641
Estimating biases using als...
RMSE: 0.9834
Estimating biases using als...
RMSE: 1.0790


1.0421233223846278

In [53]:
from surprise.model_selection import cross_validate

In [55]:
cross_validate(model, ds, cv=5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


{'test_rmse': array([1.05934384, 1.10198556, 0.97970932, 1.03911561, 1.02641121]),
 'test_mae': array([0.61162491, 0.61455908, 0.61206857, 0.61293795, 0.61737851]),
 'fit_time': (0.17432332038879395,
  0.2055954933166504,
  0.2098703384399414,
  0.20990347862243652,
  0.20276570320129395),
 'test_time': (0.33252882957458496,
  0.16189837455749512,
  0.16090083122253418,
  0.16589975357055664,
  0.1535334587097168)}

In [56]:
print(df3.shape)
print(df3.shape[0]*df3.shape[1])
print(df3.isnull().sum().sum())  # 비어있는 값이 많음.
print(df3.isnull().sum().sum()/(df3.shape[0]*df3.shape[1]))   #98%가 비어있는 값. 그래서 유사도 뽑기 어려움.

(24429, 300)
7328700
7195115
0.981772347073833


In [57]:
from surprise import KNNBasic

In [58]:
sim_options={'name':'msd'}  #오차 지정 방식: cosine, pearson ,  mse(msd) : 평균제곱차이 유사도
# sim_options={'name':'cosine'} 
# sim_options={'name':'pearson'}

# model = KNNBasic(sim_options=sim_options)  
model = KNNBasic(k=10,min_k=3, sim_options=sim_options)  # k=이웃 갯수
cross_validate(model, ds, cv=3)

#값들이 다 높아서 좋은게 아님....(비어있는 값이 너무 많음.)

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([1.09833926, 1.14270074, 1.08280275]),
 'test_mae': array([0.6608502 , 0.66475601, 0.6583568 ]),
 'fit_time': (45.956180572509766, 39.00291061401367, 38.250988245010376),
 'test_time': (21.03840684890747, 23.665138959884644, 19.306752681732178)}

In [59]:
from surprise import SVD   #차원을 줄여주는 방식.

In [60]:
svd = SVD(n_factors=50) #속성 50개로 조정
svd

In [62]:
cross_validate(svd, ds)

{'test_rmse': array([1.07455895, 0.98570108, 1.11203272, 1.00086312, 1.0933917 ]),
 'test_mae': array([0.6190639 , 0.61321937, 0.62972679, 0.6115843 , 0.62172608]),
 'fit_time': (4.205684185028076,
  4.280467510223389,
  4.282533884048462,
  4.236926317214966,
  4.245129585266113),
 'test_time': (0.2500283718109131,
  2.3852546215057373,
  0.23209238052368164,
  0.24886679649353027,
  0.2477867603302002)}

In [65]:
trainset = ds.build_full_trainset()   # 전체데이터를 훈련데이터로 사용

In [66]:
svd.fit(trainset)

In [67]:
testset = trainset.build_anti_testset() #값이 없는 데이터 추출

In [68]:
print(type(testset))
print(len(testset))

<class 'list'>
7195115


In [69]:
predict = svd.test(testset)

In [70]:
print(type(predict))
print(len(predict))
predict[:10]

<class 'list'>
7195115


[Prediction(uid=0, iid=2, r_ui=1.5830969045925816, est=2.0758378614162916, details={'was_impossible': False}),
 Prediction(uid=0, iid=23, r_ui=1.5830969045925816, est=1.8166140491689664, details={'was_impossible': False}),
 Prediction(uid=0, iid=25, r_ui=1.5830969045925816, est=1.624260876188478, details={'was_impossible': False}),
 Prediction(uid=0, iid=29, r_ui=1.5830969045925816, est=2.0881647080308072, details={'was_impossible': False}),
 Prediction(uid=0, iid=61, r_ui=1.5830969045925816, est=1.540195289739747, details={'was_impossible': False}),
 Prediction(uid=0, iid=68, r_ui=1.5830969045925816, est=3.365183505853534, details={'was_impossible': False}),
 Prediction(uid=0, iid=86, r_ui=1.5830969045925816, est=1.4177973027338004, details={'was_impossible': False}),
 Prediction(uid=0, iid=107, r_ui=1.5830969045925816, est=1.686639864765416, details={'was_impossible': False}),
 Prediction(uid=0, iid=111, r_ui=1.5830969045925816, est=1.782756135739636, details={'was_impossible': False

In [73]:
from collections import defaultdict   # 키가 없으면 자동으로 생성

top_n = defaultdict(list)   #키가 없으면 리스트로 기본으로 만들어 달라는 의미
for uid, iid, _, est, _ in predict:       # _ 부분은 r_ui(true_r), was_impossible 는 안불러 오겠다는 의미
    top_n[uid].append( (iid, est) )  # {사용자id: [(상품id, 상품구입개수)]} 으로 표시된다.

In [74]:
for uid, products in top_n.items():
    products.sort(key = lambda x: x[1], reverse=True)  # 사용자의 상품개수 정렬
    top_n[uid] = products[:10] # 상품구입개수 높은 상위 10개의 예측 평점

In [77]:
for idx, (uid, products) in enumerate(top_n.items()):
    if idx ==10: break
    print(uid, [product[0] for product in products])

0 [110, 68, 82, 132, 173, 0, 34, 37, 248, 41]
1 [10, 132, 110, 248, 245, 230, 34, 3, 128, 32]
2 [132, 248, 0, 41, 18, 226, 68, 27, 3, 230]
3 [132, 245, 82, 6, 37, 173, 248, 252, 3, 110]
4 [110, 132, 0, 37, 27, 248, 10, 34, 230, 32]
5 [132, 34, 110, 37, 248, 230, 32, 3, 10, 11]
6 [34, 41, 18, 248, 0, 132, 3, 110, 230, 245]
7 [34, 0, 37, 10, 132, 252, 248, 54, 3, 32]
8 [110, 34, 37, 132, 10, 245, 58, 3, 27, 32]
9 [110, 248, 10, 3, 132, 91, 11, 8, 37, 34]
